# Repurpose Millington2009 transition rules


The objectives of this notebook are:
1. Map land cover types represented in Millington 2009 to land cover types represented in AgroSuccess.
2. Identify land cover types present in Millington 2009 and not in AgroSuccess, and vice versa. 
3. Reconcile these differences to produce a succession transition table for agrosuccess: `agrosuccess_succession.csv`. This can be consumed by cymod to create a land cover transition graph. Variable and state names (rather than codes) should be used in this table.

In [1]:
import os
import sys
import warnings
from enum import Enum
import pandas as pd

# Add AgroSuccessGraph scripts to path
sys.path.insert(0, "../scripts")
from clean_millington_trans_table import (
    Succession,
    Aspect,
    SeedPresence,
    Water,
    MillingtonLct,
)

## 1. Map Millington land cover types to AgroSuccess types

In [2]:
def show_enum(e):
    line_len = max([len(item.name) for item in e])
    print(e.__name__ + "\n" + "-"*len(e.__name__))
    for item in e:
        print("{0: <{1}} {2}".format(item.name, line_len + 5, item.value))

The land cover types represented in AgroSuccess are specified by the following class:

In [3]:
class AgroSuccessLct(Enum):
    """Land cover types and corresponding codes used in AgroSuccess."""
    WATER_QUARRY = 0
    BURNT = 1
    BARLEY = 2
    WHEAT = 3
    DAL = 4
    SHRUBLAND = 5
    PINE = 6
    TRANS_FOREST = 7
    DECIDUOUS = 8
    OAK = 9   

Comparing these types to AgroSuccessLct we get the following

In [4]:
show_enum(MillingtonLct)
print("")
show_enum(AgroSuccessLct)

MillingtonLct
-------------
PINE                    1
TRANSITION_FOREST       2
DECIDUOUS               3
HOLM_OAK                4
PASTURE                 5
HOLM_OAK_W_PASTURE      6
CROPLAND                7
SCRUBLAND               8
WATER_QUARRY            9
URBAN                   10
BURNT                   11

AgroSuccessLct
--------------
WATER_QUARRY      0
BURNT             1
BARLEY            2
WHEAT             3
DAL               4
SHRUBLAND         5
PINE              6
TRANS_FOREST      7
DECIDUOUS         8
OAK               9


The simple 1:1 mappings are as follows:

In [5]:
m_to_a_lct = {
    MillingtonLct.PINE: AgroSuccessLct.PINE,
    MillingtonLct.HOLM_OAK: AgroSuccessLct.OAK,
    MillingtonLct.DECIDUOUS: AgroSuccessLct.DECIDUOUS,
    MillingtonLct.WATER_QUARRY: AgroSuccessLct.WATER_QUARRY,
    MillingtonLct.BURNT: AgroSuccessLct.BURNT,
    MillingtonLct.TRANSITION_FOREST: AgroSuccessLct.TRANS_FOREST,    
}

In [6]:
unmapped_m_lcts = [lct.name for lct in MillingtonLct 
                   if lct not in m_to_a_lct.keys()]
print("LCTs in Millington, not used in AgroSuccess:", 
      unmapped_m_lcts, sep="\n")
print()
unmapped_as_lcts = [lct.name for lct in AgroSuccessLct 
                    if lct not in m_to_a_lct.values()]
print("LCTs in AgroSuccess, not used in Millington:", 
      unmapped_as_lcts, sep="\n")

LCTs in Millington, not used in AgroSuccess:
['PASTURE', 'HOLM_OAK_W_PASTURE', 'CROPLAND', 'SCRUBLAND', 'URBAN']

LCTs in AgroSuccess, not used in Millington:
['BARLEY', 'WHEAT', 'DAL', 'SHRUBLAND']


These results suggest the following modifications to repurpose the Millington 2009 transition rules for AgroSuccess:

- Pasture and scrubland -> shrubland
- cropland differentiated into wheat, barley and depleated agricultural land

In [7]:
m_table_file = os.path.join("tmp", "millington_succession.csv")
m_df = pd.read_csv(m_table_file)
print(m_df.head())

   start  succession  aspect  pine  oak  deciduous  water  delta_D  delta_T
0      1           1       0     1    0          0      0        1        0
1      1           1       0     1    0          0      1        1        0
2      1           1       0     1    0          0      2        1        0
3      1           1       1     1    0          0      0        1        0
4      1           1       1     1    0          0      1        1        0


To help investigate further, let's look at the Millington table with state codes replaced with numbers

In [8]:
def get_translator(trans_enum):
    """Make a function which uses an enum to translate a dataframe column."""
    def code_translator(df, col_name, post_proc_func=None):
        """Replace codes with names in the :obj:`pandas.DataFrame` column.
        
        Args:
            df (:obj:`pandas.DataFrame`): Datafame containing column to 
                convert.
            col_name (str): Name of the column to convert.
            post_proc_func (function, optional): Function to apply to each 
                enum member name after it's been used to replace a codes in 
                the column.
        """
        df[col_name] = df[col_name].apply(lambda x: trans_enum(x).name)
        if post_proc_func:
            df[col_name] = df[col_name].apply(post_proc_func)
        return df
    return code_translator

def millington_trans_table_codes_to_names(df):
    """Replace state/condition codes in Millington trans table with names."""
    for state_col in ["start", "delta_D"]:
        df = get_translator(MillingtonLct)(df, state_col, lambda x: x.lower())

    for seed_col in ["pine", "oak", "deciduous"]:
        df = get_translator(SeedPresence)(
            df, seed_col, lambda x: True if x=="true" else False)

    df = get_translator(Succession)(df, "succession", lambda x: x.lower())
    df = get_translator(Aspect)(df, "aspect", lambda x: x.lower())
    df = get_translator(Water)(df, "water", lambda x: x.lower())
    return df

In [9]:
m_df = millington_trans_table_codes_to_names(m_df)
print(m_df.head())

  start succession aspect   pine    oak  deciduous   water delta_D  delta_T
0  pine  secondary  north  False  False      False   xeric    pine        0
1  pine  secondary  north  False  False      False   mesic    pine        0
2  pine  secondary  north  False  False      False  hydric    pine        0
3  pine  secondary  south  False  False      False   xeric    pine        0
4  pine  secondary  south  False  False      False   mesic    pine        0


### 1. Drop URBAN and HOLM_OAK_W_PASTURE
 The `URBAN` and `HOLM_OAK_W_PASTURE` land cover types used in Millington 2009 are not needed in AgroSuccess so should be removed entirely. To ensure model integrity I will check that there are no land cover types which *only* come about by transition *from* `URBAN` or `HOLM_OAK_W_PASTURE`. I.e.

In [10]:
def state_is_exclusive_source_of_other_state(trans_df, state_name):
    """True if at least one state only accessible from `state_name`."""
    def tgt_states(df, src_lct_name):
        """Get the states which can originate from `src_lct_name`.

        Exclude the `src_lct_name` state itself.

        Returns:
            list: Names of states which have `src_lct_name` as their source.
        """
        start_col = "start"
        end_col = "delta_D"
        # TODO add warning if src state not in dataframe
        all_trans = df.groupby(by=[start_col, end_col]).size().reset_index()
        if len(all_trans[all_trans[start_col] == src_lct_name]) == 0:
            warnings.warn("No start state called '{0}'".format(src_lct_name))
            return []
        else:
            tgt_trans = all_trans[(all_trans[start_col] == src_lct_name) 
                                  & (all_trans[end_col] != src_lct_name)]
            return list(tgt_trans[end_col].values)    

    def src_states(df, tgt_lct_name):
        """Get the states which `tgt_lct_name` can transition from.

        Exclude the `tgt_lct_name` state itself.

        Returns:
            list: Names of states which can transition to `tgt_lct_name`.    
        """
        start_col = "start"
        end_col = "delta_D"
        all_trans = df.groupby(by=[start_col, end_col]).size().reset_index()
        src_trans = all_trans[(all_trans[end_col] == tgt_lct_name) 
                              & (all_trans[start_col] != tgt_lct_name)]
        return list(src_trans[start_col].values)
    
    states_from_state_name = tgt_states(trans_df, state_name)
    exclusive_source_for = []
    for other_state in states_from_state_name:
        other_state_sources = src_states(trans_df, other_state)
        other_state_sources.remove(state_name)
        if len(other_state_sources) < 1:
            exclusive_source_for.append(other_state)
    if exclusive_source_for:
        print("{0} is the only source for states: {1}".format(
            state_name, ", ".join(exclusive_source_for)))
        return True
    else:
        return False

In [11]:
def drop_holm_oak_w_pasture_and_urban(df):
    """Remove rows with excluded land cover types as start or end state.
    
    holm_oak_w_pasture and urban land cover types removed.
    """
    def row_excludes_lct(row, lct_name):
        """Return True if row doesn't have lct as start or end state."""
        start_col = "start"
        end_col = "delta_D"
        if row[start_col] == lct_name or row[end_col] == lct_name:
            return False
        else:
            return True
    
    # Confirm removing these states won't leave any other states in the model
    # inaccessbile, and remove it.
    for state in ["holm_oak_w_pasture", "urban"]:
        assert state_is_exclusive_source_of_other_state(df, state) == False
        no_rows = len(df.index)
        df = df[df.apply(lambda x: row_excludes_lct(x, state), axis=1)]
        assert len(df.index) < no_rows   
    return df

In [12]:
as_df = drop_holm_oak_w_pasture_and_urban(m_df)

Outstanding queries about mapping from Millington LCTs to AgroSuccess LCTs
1. How should I Map CROPLAND to BARLEY, WHEAT and DAL?
2. Why doesn't shrubland appear in MillingtonLct? shrubland appears in the Millington2009 paper but not in James's thesis. I seem to remember discussing whether that might map directly to another landcover type in the thesis. Possibly pasture or scrubland?
3. Might it make sense to drop HOLM_OAK_WITH_PASTURE, URBAN, PASTURE, SCRUBLAND?

The problem is what to do about transitions going from another state to cropland in Millington2009. In that case, which of my new type (Barley, Wheat or DAL) should the cell transition to?

**Query**: what do land cover classes 7, 8, 10 and 11 given in supplementary materials correspond to? They all transition *to* shrubland (land cover class 5) an so are similar to the 'burnt' class but differ in succession pathway and duration of time spent in class before transition to shrubland.

### 2. Replace 'cropland' with 'barley', 'wheat' and 'DAL'

Find states transitioning from cropland to something else:

In [13]:
as_df[as_df["start"] == "cropland"]["delta_D"].unique()

array(['pasture'], dtype=object)

So all cropland transitions to pasture. What transitions to cropland?

In [14]:
as_df[as_df["delta_D"] == "cropland"]["start"].unique()

array([], dtype=object)

Nothing. Okay so although `MillingtonLct.CROPLAND` doesn't map 1:1 with anything in `AgroSuccessLct`, we can replace rows matching  `as_df["start"] == "cropland"` with the same conditions starting with `"wheat"`, `barley` or `"dal"`

In [15]:
def replace_cropland_with_new_crop_types(df):
    """Replace Millington's cropland state with wheat, barley and DAL.
    
    Args:
        df (:obj:`pandas.DataFrame`): Original transition table containing 
            'cropland' as a land cover state.
            
    Returns:
        df: A new dataframe where rows representing transitions involving 
            cropland are replaced with rows describing transitions involving
            wheat, barley and DAL (depleated agricultural land) states.
    """
    # There are no transitions where cropland is the target state. 
    # Correspondingly no transitions have the new cropland land cover types
    # as their target state. This makes sense, as cropland is something which
    # humans need to create.
    assert len(df[df["delta_D"] == "cropland"].index) == 0
    
    # Rows from old table where cropland is the transition's starting state
    from_cropland = df[df["start"] == "cropland"]
    
    new_crop_dfs = []
    for crop in ["wheat", "barley", "dal"]:
        new_crop = from_cropland.copy()
        new_crop.loc[:,"start"] = crop
        new_crop_dfs.append(new_crop)

    new_df = df.copy()
    new_df = new_df[new_df["start"] != "cropland"] # remove old cropland rows
    new_df = pd.concat([new_df] + new_crop_dfs)

    assert len(new_df.index) == len(df.index) - len(from_cropland.index) \
                                + 3*len(from_cropland.index)    
    
    return new_df

In [16]:
as_df = replace_cropland_with_new_crop_types(as_df)

In [17]:
len(as_df.index)

676

In [19]:
as_df.head()

,start,succession,aspect,pine,oak,deciduous,water,delta_D,delta_T
0,pine,secondary,north,False,False,False,xeric,pine,0
1,pine,secondary,north,False,False,False,mesic,pine,0
2,pine,secondary,north,False,False,False,hydric,pine,0
3,pine,secondary,south,False,False,False,xeric,pine,0
4,pine,secondary,south,False,False,False,mesic,pine,0


### 3. Pasture and scrubland -> shrubland

## Interpreting land cover classes

The land cover classes whose transitions are specified in the table are described in James Millington's PhD thesis. They are described as follows:

In [ ]:
state_vals = pd.DataFrame({
    'state_num': range(1,12),
    'Land-cover': ['Pine', 'Transition Forest', 'Deciduous', 
                   'Holm Oak', 'Pasture', 'Holm Oak with Pasture',
                   'Cropland', 'Shrubland', 'Water/Quarry', 'Urban',
                   'Burnt']
    })

In [ ]:
print state_vals

Mapping these to the land cover types which are relevant for my PhD we find:

In [ ]:
map_df = pd.DataFrame({'code': ['WaterQuarry', 'Burnt', 'Barley', 'Wheat', 'DAL', 'Pine', 'TransForest', 
                       'Deciduous', 'Oak', 'Shrubland'],    
                       'state_num':[9, 11, 7, 7, 7, 1, 2, 3, 4, 5]            
                      })
print map_df

Note that Barley, Wheat and DAL (Depleated Agricultural Land) all correspond to the same state in the Millington2009 model: cropland. It may be decided that this part of the model needs to be refined for my purposes, but for the time being, I will assume all three of these land cover types will behave in the same way for succession purposes

In [ ]:
print len(df.index)

In [ ]:
tmp_df = pd.merge(df, map_df, 'left', left_on='start', right_on='state_num')
tmp_df = tmp_df.rename(columns = {'code':'start_code'})
tmp_df = pd.merge(tmp_df, map_df, 'left', left_on='delta_D', right_on='state_num')
tmp_df = tmp_df.rename(columns = {'code':'end_code'})
print 'No. records before dropping rows not corresponding to PhD states: '+str(len(tmp_df.index))
tmp_df = tmp_df[(tmp_df.start_code.notnull() & tmp_df.end_code.notnull())]
print 'No. records after dropping rows not corresponding to PhD states: '+str(len(tmp_df.index))
df = tmp_df[['start_code', 'end_code', 'succession', 'aspect', 'pine', 'oak', 'deciduous', 'water', 'delta_T']]

In [ ]:
print df[['start_code', 'end_code']].drop_duplicates()

NOTE: there's a potential issue here because no state appears to transition TO Deciduous. Presumably I've dropped something important here. Something to check up on.

In [ ]:
print df.aspect.unique()